Send notif saying it started

In [2]:

import firebase_admin
from firebase_admin import credentials, messaging

cred = credentials.Certificate("./credentials.json")
firebase_admin.initialize_app(cred)



ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [7]:

topic = "News"
message = messaging.Message(
   notification=messaging.Notification(
   title="Derpy",
   body="bodyyy"
  ),topic="News" )
messaging.send(message)


'projects/flutterwatch/messages/7725488393549917356'

In [1]:
from tensorflow.python.client import device_lib
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
local_device_protos = device_lib.list_local_devices()
local_device_protos

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15982673522376018111
 xla_global_id: -1]

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:

import cv2
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Create model

In [4]:
history = []
model = tf.keras.applications.mobilenet.MobileNet()
model.summary()
base_input = model.layers[0].input
base_output = model.layers[-4].output
Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
final_output = layers.Activation('sigmoid')(final_output)
new_model = keras.Model(inputs=base_input, outputs=final_output)
print(new_model.summary())
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

Get all files that are being processed

In [5]:
training_data = []
files = []
Datadirectory = './input/'
Classes = ['Closed_Eyes', 'Open_Eyes']
img_size = 224

for category in os.listdir(Datadirectory):
      path = os.path.join(Datadirectory, category)

      for img in os.listdir(path):
        files.append(os.path.join(path,img))
len(files)

84898

In [6]:
import random
random.shuffle(files)

In [7]:
files[0]

'./input/s0014\\s0014_02877_0_1_0_0_1_01.png'

In [8]:
batch = 200
i = 0
while batch * i < len(files):
    print("Started batch - " + str(i))
    training_data_batch = []
    end =  batch * (i+1) if batch * (i+1) < len(files) else len(files)-1
    for file in files[batch*i : end]:
           img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
           backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
           new_array = cv2.resize(backtorgb, (img_size,img_size))
           training_data_batch.append([new_array,float( file.split('_')[3])])
    X = []
    y = []
    for features, label in training_data_batch:
        X.append(features)
        y.append(label)
    X = np.array(X).reshape(-1, img_size, img_size, 3)
    X = X/255.0
    Y = np.array(y)
    print('started the model')
    history.append(new_model.fit(X, Y, epochs=4, validation_split=0.1))
    i+=1

Started batch - 0
started the model
Epoch 1/4
6/6 [==============================] - 17s 3s/step - loss: 0.4750 - accuracy: 0.7833 - val_loss: 3.2234 - val_accuracy: 0.8000
Epoch 2/4
6/6 [==============================] - 15s 3s/step - loss: 0.0584 - accuracy: 0.9722 - val_loss: 6.9976 - val_accuracy: 0.8000
Epoch 3/4
6/6 [==============================] - 14s 2s/step - loss: 0.0487 - accuracy: 0.9778 - val_loss: 7.4534 - val_accuracy: 0.8000
Epoch 4/4
6/6 [==============================] - 15s 2s/step - loss: 0.0500 - accuracy: 0.9778 - val_loss: 8.2077 - val_accuracy: 0.8000
Started batch - 1
started the model
Epoch 1/4
6/6 [==============================] - 15s 2s/step - loss: 0.2221 - accuracy: 0.9611 - val_loss: 2.7368 - val_accuracy: 0.8500
Epoch 2/4
6/6 [==============================] - 15s 2s/step - loss: 0.1979 - accuracy: 0.9556 - val_loss: 1.0826 - val_accuracy: 0.9000
Epoch 3/4
6/6 [==============================] - 15s 2s/step - loss: 0.0452 - accuracy: 0.9889 - val_loss:

In [12]:
new_model.save("my_new_model.h5")